In [1]:
#!pip install requests beautifulsoup4

In [2]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.edge.options import Options as EdgeOptions
from selenium.webdriver.common.by import By
import time
import os
from selenium import webdriver
from datetime import datetime


In [3]:

# Set up Edge options
edge_options = EdgeOptions()
edge_options.add_argument("headless")  # Remove this line if you want to see the browser

# Path to Edge WebDriver
service = EdgeService(r"C:\Users\alons\Downloads\Installers\edgedriver_win64\msedgedriver.exe")

# Start the WebDriver
driver = webdriver.Edge(service=service, options=edge_options)

# Base URL template
base_url = (
    "https://auctions.royaltyexchange.com/orderbook/open-listings/"
    "?filter{{state.in}}=open&filter{{state.in}}=upcoming&filter{{state.in}}=preview"
    "&filter{{state.in}}=listed&filter{{state.in}}=live&only_favorited=0&only_offered=0"
    "&only_user_active_offers=0&sort[]=-is_open_auction&sort[]=-close_date"
    "&sort[]=featured_position&sort[]=-if_new_published_date"
    "&sort[]=-owner_last_active_date&sort[]=-published_date&page={}&page_size=50"
)

# Load page 1 to detect number of pages
driver.get(base_url.format(1))
time.sleep(2)

# Detect pagination. This element holds a unordered list with the total number of pages
pagination_buttons = driver.find_elements(By.CSS_SELECTOR, "ul.MuiPagination-ul > li button")


# Determine total number of pages dynamically
page_numbers = []
for btn in pagination_buttons:
    try:
        num = int(btn.text)
        page_numbers.append(num)
    except ValueError:
        continue  # ignore non-numeric buttons

last_page = max(page_numbers) if page_numbers else 1

# Collect pages
all_pages_html = ""

for page_num in range(1, last_page + 1):
    print(f"Fetching page {page_num}...")
    driver.get(base_url.format(page_num))
    time.sleep(2)
    all_pages_html += driver.page_source + "\n\n" + "=" * 100 + "\n\n"

driver.quit()

# Save in the current working directory
output_path = os.path.join(os.getcwd(), "royaltyexchange_pages.txt")

with open(output_path, "w", encoding="utf-8") as f:
    f.write(all_pages_html)

print(f"✅ Done! File saved to: {output_path}")


Fetching page 1...
Fetching page 2...
Fetching page 3...
Fetching page 4...
Fetching page 5...
Fetching page 6...
Fetching page 7...
Fetching page 8...
Fetching page 9...
Fetching page 10...
Fetching page 11...
Fetching page 12...
Fetching page 13...
✅ Done! File saved to: c:\Users\alons\OneDrive - Cornell University\Cornell University\Spring 2025\CFEM Research Assistant\Parser\royalty_scraper\royaltyexchange_pages.txt


In [4]:

# Load content from file
with open("royaltyexchange_pages.txt", "r", encoding="utf-8") as f:
    new_content = f.read()

# Extract all numbers that follow "ID:"
new_ids = re.findall(r'ID:\s*(\d+)', new_content)

# Create a DataFrame
df_new_ids = pd.DataFrame(new_ids, columns=['Listing ID'])

# Generate date string
today_str = datetime.today().strftime('%Y-%m-%d')

# Save to CSV with date
filename = f"new_extracted_listing_ids_{today_str}.csv"
df_new_ids.to_csv(filename, index=False)

print(f"Saved to {filename}")

Saved to new_extracted_listing_ids_2025-04-05.csv
